In [ ]:
# Install required libraries
!pip install -q opencv-python-headless

# Import standard libraries
import os
import cv2
import torch
import numpy as np
import pandas as pd
from torchvision import models, transforms
from PIL import Image
from tqdm import tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ✅ Updated base path pointing to your segmented folders
BASE_PATH = "/content/drive/MyDrive/segmented"

Mounted at /content/drive


In [ ]:
import os

print("Checking BASE_PATH:", BASE_PATH)
if os.path.exists(BASE_PATH):
    folders = os.listdir(BASE_PATH)
    print(f"✅ Found {len(folders)} folders in BASE_PATH.")
    print("Sample folder names:", folders[:3])
else:
    print("❌ BASE_PATH does not exist. Check the path or re-mount your drive.")


Checking BASE_PATH: /content/drive/MyDrive/segmented
✅ Found 135 folders in BASE_PATH.
Sample folder names: ['chicken_sessions.csv', '20250721_114639_Monday_chicken-141_batch-32_chicken-age-32', '20250721_114526_Monday_chicken-140_batch-32_chicken-age-32']


In [ ]:
# Load pretrained ResNet50 and remove FC layers to get 2048-dim output
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1])  # Remove FC layers
resnet.to(device)
resnet.eval()

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s]


Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)


In [ ]:
# Preprocessing pipeline: normalize depth → JET map → resize → normalize for ResNet
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

def preprocess_depth_image(path):
    try:
        depth = cv2.imread(path, cv2.IMREAD_UNCHANGED)
        depth = cv2.normalize(depth, None, 0, 255, cv2.NORM_MINMAX).astype(np.uint8)
        jet = cv2.applyColorMap(depth, cv2.COLORMAP_JET)
        return transform(jet)
    except Exception as e:
        print(f"Error in preprocessing {path}: {e}")
        return None


In [ ]:
output_rows = []

# Get all folders in the base directory
chicken_folders = sorted([
    d for d in os.listdir(BASE_PATH)
    if os.path.isdir(os.path.join(BASE_PATH, d))
])

print(f"🔍 Total folders (chickens/days) found: {len(chicken_folders)}")

for folder in tqdm(chicken_folders, desc="Processing Folders"):
    chicken_path = os.path.join(BASE_PATH, folder)

    # Get all .png images (case-insensitive)
    image_files = sorted([
        f for f in os.listdir(chicken_path)
        if f.lower().endswith(".png")
    ])

    print(f"\n📂 {folder} → Found {len(image_files)} PNG images")

    if len(image_files) == 0:
        print(f"⚠️ Skipping {folder} — No .png images found.")
        continue

    for img_file in image_files:
        img_path = os.path.join(chicken_path, img_file)

        # Preprocess depth image
        img_tensor = preprocess_depth_image(img_path)
        if img_tensor is None:
            print(f"❌ Skipped {img_file} in {folder} — Preprocessing failed.")
            continue

        img_tensor = img_tensor.unsqueeze(0).to(device)

        try:
            with torch.no_grad():
                features = resnet(img_tensor).squeeze().cpu().numpy()
            print(f"✅ Extracted features from {img_file} in {folder}")
        except Exception as e:
            print(f"❌ Error extracting features from {img_file} in {folder}: {e}")
            continue

        # Store metadata + feature vector
        row = [folder, img_file] + features.tolist()
        output_rows.append(row)

# Summary after loop
print("\n📊 Feature Extraction Summary:")
print(f"✅ Total folders processed: {len(chicken_folders)}")
print(f"✅ Total images processed: {len(output_rows)}")
if len(output_rows) > 0:
    print(f"✅ Sample row: {output_rows[0][:5]}")
else:
    print("❌ No features extracted. Check logs above.")


🔍 Total folders (chickens/days) found: 133


Processing Folders:   0%|          | 0/133 [00:00<?, ?it/s]


📂 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23 → Found 8 PNG images
✅ Extracted features from rgb_20250712_105832_715015_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105835_034034_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105837_126723_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105841_967950_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105843_945226_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105852_571966_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105855_417582_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23


Processing Folders:   1%|          | 1/133 [00:08<18:44,  8.52s/it]

✅ Extracted features from rgb_20250712_105857_082772_instance-0.png in 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23

📂 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23 → Found 13 PNG images
✅ Extracted features from rgb_20250712_105956_066403_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105957_237449_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_105958_998218_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_110000_113100_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_110001_390137_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extracted features from rgb_20250712_110002_672555_instance-0.png in 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23
✅ Extr

...

✅ Extracted features from rgb_20250721_114610_969820_instance-0.png in 20250721_114526_Monday_chicken-140_batch-32_chicken-age-32

📂 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32 → Found 19 PNG images
✅ Extracted features from rgb_20250721_114651_766327_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114653_366005_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114658_741842_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114659_908571_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114703_267007_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114703_267007_instance-1.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32
✅ Extr

Processing Folders:  99%|█████████▉| 132/133 [23:29<00:14, 14.51s/it]

✅ Extracted features from rgb_20250721_114719_660703_instance-0.png in 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32

📂 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32 → Found 5 PNG images
✅ Extracted features from rgb_20250721_114754_315564_instance-0.png in 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114755_603016_instance-0.png in 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114757_597159_instance-0.png in 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32
✅ Extracted features from rgb_20250721_114759_843131_instance-0.png in 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32


Processing Folders: 100%|██████████| 133/133 [23:34<00:00, 10.64s/it]

✅ Extracted features from rgb_20250721_114802_268593_instance-0.png in 20250721_114738_Monday_chicken-142_batch-32_chicken-age-32

📊 Feature Extraction Summary:
✅ Total folders processed: 133
✅ Total images processed: 1492
✅ Sample row: ['20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23', 'rgb_20250712_105832_715015_instance-0.png', 0.2315637618303299, 0.7837387323379517, 0.6818743348121643]


In [ ]:
# Define column headers: metadata + 2048 features
columns = ['Chicken_ID', 'Image_ID'] + [f'Feature_{i}' for i in range(2048)]

# Check if we have features to save
if len(output_rows) == 0:
    print("❌ No features to save. CSV not generated.")
else:
    # Create DataFrame
    df = pd.DataFrame(output_rows, columns=columns)

    # Define save path in Google Drive
    output_path = "/content/drive/MyDrive/broiler_features.csv"

    # Save the DataFrame
    df.to_csv(output_path, index=False)

    print(f"\n✅ CSV successfully saved at: {output_path}")
    print(f"🔎 Rows: {len(df)}, Columns: {len(df.columns)}")
    print("📌 Preview:")
    display(df.head())



✅ CSV successfully saved at: /content/drive/MyDrive/broiler_features.csv
🔎 Rows: 1492, Columns: 2050
📌 Preview:


,Chicken_ID,Image_ID,Feature_0,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,...,Feature_2038,Feature_2039,Feature_2040,Feature_2041,Feature_2042,Feature_2043,Feature_2044,Feature_2045,Feature_2046,Feature_2047
0,20250712_105816_Saturday_chicken-2_batch-23_ch...,rgb_20250712_105832_715015_instance-0.png,0.231564,0.783739,0.681874,0.671735,0.423946,0.125335,0.364921,0.564257,...,0.278484,0.147399,1.011837,0.444904,0.214096,0.145934,0.201196,0.394381,0.340130,0.773048
1,20250712_105816_Saturday_chicken-2_batch-23_ch...,rgb_20250712_105835_034034_instance-0.png,0.227791,0.311679,0.780369,1.097455,0.533584,0.379554,0.140341,0.647661,...,0.431649,0.309159,0.548404,0.189144,0.256014,0.265751,0.233600,0.026647,0.354563,1.078711
2,20250712_105816_Saturday_chicken-2_batch-23_ch...,rgb_20250712_105837_126723_instance-0.png,0.140049,0.771959,0.575042,0.526495,0.687173,0.764766,0.382077,0.548400,...,0.038213,0.060250,0.341898,0.149625,0.489923,0.194420,0.279601,0.249994,0.324571,0.718451
3,20250712_105816_Saturday_chicken-2_batch-23_ch...,rgb_20250712_105841_967950_instance-0.png,0.111779,0.665938,0.461407,0.517627,0.271816,0.270046,0.362527,0.414530,...,0.267259,0.434148,0.527527,0.134241,0.349515,0.291169,0.632144,0.346420,0.345855,0.928557
4,20250712_105816_Saturday_chicken-2_batch-23_ch...,rgb_20250712_105843_945226_instance-0.png,0.153487,1.009318,0.510623,0.187988,0.489786,0.192919,0.477994,0.284584,...,0.070210,0.416563,0.559865,0.223223,0.206875,0.218620,0.310413,0.092263,0.307174,1.277710


In [ ]:
# Cell 1: Imports and Setup
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from typing import Dict, List, Tuple
import glob
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [ ]:
def extract_2d_features_from_image(png_path: str) -> Dict[str, float]:
    """Extract 2D geometric features from PNG image."""
    try:
        # Preprocess image and get contour
        image, binary, contour = preprocess_image(png_path)

        # Extract geometric features
        features_2d = extract_geometric_features(contour, pixel_to_mm=1.0)

        return features_2d

    except Exception as e:
        print(f"Error extracting 2D features from {png_path}: {e}")
        # Return default values if extraction fails
        return {
            'projected_area': 0.0,
            'perimeter': 0.0,
            'width': 0.0,
            'height': 0.0,
            'convex_hull_area': 0.0,
            'minor_axis_length': 0.0,
            'major_axis_length': 0.0,
            'eccentricity': 0.0,
            'convex_hull_perimeter': 0.0,
            'approx_area': 0.0,
            'approx_perimeter': 0.0,
            'area_ratio_rect': 0.0,
            'area_ratio_hull': 0.0,
            'max_convexity_defect': 0.0,
            'sum_convexity_defects': 0.0,
            'equiv_diameter': 0.0
        }


In [ ]:
def preprocess_image(image_path: str) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Preprocess image and extract largest contour.
    Returns:
        Tuple of (original_image, binary_image, largest_contour)
    """
    import cv2
    import numpy as np

    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Could not read image at {image_path}")

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    binary = cv2.adaptiveThreshold(
        blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
        cv2.THRESH_BINARY_INV, 11, 2
    )

    kernel = np.ones((3, 3), np.uint8)
    binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)

    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        raise ValueError("No contours found in the image")

    largest_contour = max(contours, key=cv2.contourArea)

    return image, binary, largest_contour


In [ ]:
def extract_3d_features_from_depth(npy_path: str) -> Dict[str, float]:
    """Extract 3D features from depth data."""
    import numpy as np

    try:
        depth_data = np.load(npy_path)
        mask = (depth_data > 0).astype(np.uint8)
        segmented_depth = depth_data[mask > 0]

        if segmented_depth.size == 0:
            raise ValueError("No valid depth pixels found.")

        max_d = np.max(segmented_depth)
        min_d = np.min(segmented_depth)
        avg_d = np.mean(segmented_depth)
        std_d = np.std(segmented_depth)
        sum_d = np.sum(segmented_depth)
        range_d = max_d - min_d
        min_minus_avg = np.abs(min_d - avg_d)
        max_minus_avg = np.abs(max_d - avg_d)
        projected_area = segmented_depth.size
        approx_volume = projected_area * max_d - sum_d

        return {
            'Feature17_ApproxVolume': float(approx_volume),
            'Feature18_MaxDepth': float(max_d),
            'Feature19_MinDepth': float(min_d),
            'Feature20_AvgDepth': float(avg_d),
            'Feature21_DepthRange': float(range_d),
            'Feature22_StdDepth': float(std_d),
            'Feature23_SumDepth': float(sum_d),
            'Feature24_MinMinusAvg': float(min_minus_avg),
            'Feature25_MaxMinusAvg': float(max_minus_avg)
        }

    except Exception as e:
        print(f"Error extracting 3D features from {npy_path}: {e}")
        return {
            'Feature17_ApproxVolume': 0.0,
            'Feature18_MaxDepth': 0.0,
            'Feature19_MinDepth': 0.0,
            'Feature20_AvgDepth': 0.0,
            'Feature21_DepthRange': 0.0,
            'Feature22_StdDepth': 0.0,
            'Feature23_SumDepth': 0.0,
            'Feature24_MinMinusAvg': 0.0,
            'Feature25_MaxMinusAvg': 0.0
        }


In [ ]:
import cv2
import numpy as np

def point_to_line_distance(point, line_start, line_end):
    x0, y0 = point
    x1, y1 = line_start
    x2, y2 = line_end
    num = abs((y2 - y1) * x0 - (x2 - x1) * y0 + x2 * y1 - y2 * x1)
    den = np.sqrt((y2 - y1)**2 + (x2 - x1)**2)
    return num / den if den != 0 else np.linalg.norm(np.array(point) - np.array(line_start))

def douglas_peucker(points, epsilon):
    if len(points) <= 2:
        return points
    max_dist, max_idx = max((point_to_line_distance(points[i], points[0], points[-1]), i) for i in range(1, len(points) - 1))
    if max_dist > epsilon:
        return douglas_peucker(points[:max_idx+1], epsilon)[:-1] + douglas_peucker(points[max_idx:], epsilon)
    return [points[0], points[-1]]

def calculate_convexity_defects(contour):
    if contour is None or len(contour) <= 3:
        return (0.0, 0.0)
    try:
        hull = cv2.convexHull(contour, returnPoints=False)
        if len(hull) <= 3:
            return (0.0, 0.0)
        defects = cv2.convexityDefects(contour, hull)
        if defects is None:
            return (0.0, 0.0)
        depths = defects[:, 0, 3] / 256.0
        return float(np.max(depths)), float(np.sum(depths))
    except Exception as e:
        print(f"Warning: Error calculating convexity defects: {e}")
        return (0.0, 0.0)

def extract_geometric_features(contour, pixel_to_mm=1.0):
    features = {}
    area = cv2.contourArea(contour) * (pixel_to_mm ** 2)
    features['projected_area'] = float(area)
    perimeter = cv2.arcLength(contour, True) * pixel_to_mm
    features['perimeter'] = float(perimeter)
    x, y, w, h = cv2.boundingRect(contour)
    features['width'] = float(w * pixel_to_mm)
    features['height'] = float(h * pixel_to_mm)
    hull = cv2.convexHull(contour)
    hull_area = cv2.contourArea(hull) * (pixel_to_mm ** 2)
    features['convex_hull_area'] = float(hull_area)
    if len(contour) >= 5:
        (_, (major, minor), _) = cv2.fitEllipse(contour)
        features['minor_axis_length'] = float(minor * pixel_to_mm)
        features['major_axis_length'] = float(major * pixel_to_mm)
        features['eccentricity'] = float(np.sqrt(1 - (minor / major) ** 2)) if major > 0 else 0.0
    else:
        features['minor_axis_length'] = 0.0
        features['major_axis_length'] = 0.0
        features['eccentricity'] = 0.0
    hull_perimeter = cv2.arcLength(hull, True) * pixel_to_mm
    features['convex_hull_perimeter'] = float(hull_perimeter)
    epsilon = 0.01 * perimeter
    approx = douglas_peucker([(p[0][0], p[0][1]) for p in contour], epsilon)
    approx_cnt = np.array([[[int(x), int(y)]] for x, y in approx], dtype=np.int32)
    features['approx_area'] = float(cv2.contourArea(approx_cnt) * (pixel_to_mm ** 2))
    features['approx_perimeter'] = float(cv2.arcLength(approx_cnt, True) * pixel_to_mm)
    rect_area = w * h * (pixel_to_mm ** 2)
    features['area_ratio_rect'] = float(area / rect_area) if rect_area > 0 else 0.0
    features['area_ratio_hull'] = float(area / hull_area) if hull_area > 0 else 0.0
    max_defect, sum_defects = calculate_convexity_defects(contour)
    features['max_convexity_defect'] = float(max_defect * pixel_to_mm)
    features['sum_convexity_defects'] = float(sum_defects * pixel_to_mm)
    features['equiv_diameter'] = float(2 * np.sqrt(area / np.pi))
    features['approx_volume'] = float(area * np.sqrt(area / np.pi) / 3)
    return features


def extract_3d_features(segmented_depth, mask):
    valid_depths = segmented_depth[segmented_depth > 0]  # ✅ Define first

    if valid_depths.size == 0:
        min_d = max_d = avg_d = std_d = sum_d = depth_range = min_minus_avg = max_minus_avg = 0.0
    else:
        min_d = np.min(valid_depths)
        max_d = np.max(valid_depths)
        avg_d = np.mean(valid_depths)
        std_d = np.std(valid_depths)
        sum_d = np.sum(valid_depths)
        depth_range = max_d - min_d
        min_minus_avg = abs(min_d - avg_d)
        max_minus_avg = abs(max_d - avg_d)

    projected_area = np.count_nonzero(mask)
    approx_volume = projected_area * max_d - sum_d

    return {
        "Feature17_ApproxVolume": approx_volume,
        "Feature18_MaxDepth": max_d,
        "Feature19_MinDepth": min_d,
        "Feature20_AvgDepth": avg_d,
        "Feature21_DepthRange": depth_range,
        "Feature22_StdDepth": std_d,
        "Feature23_SumDepth": sum_d,
        "Feature24_MinMinusAvg": min_minus_avg,
        "Feature25_MaxMinusAvg": max_minus_avg,
    }



In [ ]:
# Cell 3: Processing Folders and Extracting Features
def get_chicken_info_from_path(folder_path: str) -> Dict[str, str]:
    folder_name = os.path.basename(folder_path)
    parts = folder_name.split('_')
    try:
        date_str = f"{parts[0]}_{parts[1]}"
        day_of_week = parts[2]
        time_str = parts[3]
        chicken_batch_age = '_'.join(parts[4:])
        chicken_id = chicken_batch_age.split('chicken-')[1].split('_')[0]
        batch_id = chicken_batch_age.split('batch-')[1].split('_')[0]
        chicken_age = chicken_batch_age.split('chicken-age-')[1]
        return {'date': date_str, 'day_of_week': day_of_week, 'time': time_str, 'chicken_id': chicken_id, 'batch_id': batch_id, 'chicken_age': chicken_age, 'folder_name': folder_name}
    except Exception as e:
        print(f"Error parsing folder name '{folder_name}': {e}")
        return {'date': 'unknown', 'day_of_week': 'unknown', 'time': 'unknown', 'chicken_id': 'unknown', 'batch_id': 'unknown', 'chicken_age': 'unknown', 'folder_name': folder_name}

def find_matching_files(chicken_folder: str) -> List[Tuple[str, str]]:
    png_files = glob.glob(os.path.join(chicken_folder, "*.png"))
    npy_files = glob.glob(os.path.join(chicken_folder, "*.npy"))
    png_dict = {os.path.splitext(os.path.basename(f))[0]: f for f in png_files}
    npy_dict = {os.path.splitext(os.path.basename(f))[0]: f for f in npy_files}
    return [(png_dict[base], npy_dict[base]) for base in png_dict if base in npy_dict]

def process_chicken_folder(chicken_folder: str) -> List[Dict]:
    chicken_info = get_chicken_info_from_path(chicken_folder)
    matching_pairs = find_matching_files(chicken_folder)
    features_list = []
    for png_path, npy_path in matching_pairs:
        try:
            features_2d = extract_2d_features_from_image(png_path)
            features_3d = extract_3d_features_from_depth(npy_path)
            combined_features = {**features_2d, **features_3d, 'png_path': png_path, 'npy_path': npy_path}
            features_list.append(combined_features)
        except Exception as e:
            print(f"Error processing {png_path}: {e}")
            continue
    return features_list


In [ ]:
# Cell 4: Process All Chicken Folders and Extract Features
def process_chicken_folder(chicken_folder: str) -> List[Dict]:
    chicken_info = get_chicken_info_from_path(chicken_folder)
    matching_pairs = find_matching_files(chicken_folder)
    features_list = []
    for png_path, npy_path in matching_pairs:
        try:
            features_2d = extract_2d_features_from_image(png_path)
            features_3d = extract_3d_features_from_depth(npy_path)
            combined_features = {**features_2d, **features_3d, 'png_path': png_path, 'npy_path': npy_path}
            features_list.append(combined_features)
        except Exception as e:
            print(f"Error processing {png_path}: {e}")
            continue
    return features_list


In [ ]:
# Cell 5: Extract Features from All Chicken Folders
def main():
    segmented_dir = "/content/drive/MyDrive/segmented"  # Path to the segmented folder
    if not os.path.exists(segmented_dir):
        print(f"Error: {segmented_dir} directory not found!")
        return

    chicken_folders = [f for f in os.listdir(segmented_dir) if os.path.isdir(os.path.join(segmented_dir, f))]

    print(f"Found {len(chicken_folders)} chicken folders to process")

    all_features = []

    for chicken_folder in tqdm(chicken_folders, desc="Processing chicken folders"):
        folder_path = os.path.join(segmented_dir, chicken_folder)
        try:
            features_list = process_chicken_folder(folder_path)
            all_features.extend(features_list)
            print(f"Processed {chicken_folder}: {len(features_list)} instances")
        except Exception as e:
            print(f"Error processing folder {chicken_folder}: {e}")
            continue

    if all_features:
        df = pd.DataFrame(all_features)
        output_csv = "/content/drive/MyDrive/all_features_clean.csv"
        df.to_csv(output_csv, index=False)

        print(f"Feature extraction completed!")
        print(f"Output saved to: {output_csv}")
    else:
        print("No features were extracted!")

# Run the main function
main()


Found 133 chicken folders to process


Processing chicken folders:   1%|          | 1/133 [00:15<33:48, 15.37s/it]

Processed 20250721_114639_Monday_chicken-141_batch-32_chicken-age-32: 19 instances


Processing chicken folders:   2%|▏         | 2/133 [00:27<29:14, 13.39s/it]

Processed 20250721_114526_Monday_chicken-140_batch-32_chicken-age-32: 16 instances


...

Processed 20250712_110040_Saturday_chicken-4_batch-23_chicken-age-23: 8 instances


Processing chicken folders:  99%|█████████▉| 132/133 [19:01<00:12, 12.30s/it]

Processed 20250712_105946_Saturday_chicken-3_batch-23_chicken-age-23: 13 instances


Processing chicken folders: 100%|██████████| 133/133 [19:07<00:00,  8.63s/it]

Processed 20250712_105816_Saturday_chicken-2_batch-23_chicken-age-23: 8 instances
Feature extraction completed!
Output saved to: /content/drive/MyDrive/all_features_clean.csv


In [ ]:
# Cell 5: Merging Extracted Features with 2048D Features
# Load the 2048D features CSV
fused_df = pd.read_csv("/content/drive/MyDrive/fused_features.csv")

# Load the 2D+3D features CSV
features_2d_3d_df = pd.read_csv("/content/drive/MyDrive/all_features_clean.csv")

# Merge based on Image_ID or other relevant column
final_merged_df = pd.merge(fused_df, features_2d_3d_df, left_on='Image_ID', right_on='png_path', how='left')

# Save the merged dataset
final_output_path = "/content/drive/MyDrive/final_fused_with_2048_and_2d_3d_features.csv"
final_merged_df.to_csv(final_output_path, index=False)

print(f"Final merged file saved to: {final_output_path}")


In [ ]:
# Cell 6: Integrating Weight Data
# Load the weights CSV
weights_df = pd.read_csv("/content/drive/MyDrive/segmented/chicken_sessions.csv")  # Update with your actual weight file path

# Extract Chicken_ID from the session_path (from the weights CSV)
def extract_chicken_id_from_path(session_path):
    parts = session_path.split('\\')
    for part in parts:
        if "chicken-" in part:
            return part.split('-')[1]
    return None

weights_df['Chicken_ID_from_path'] = weights_df['session_path'].apply(extract_chicken_id_from_path)
final_merged_df['Chicken_ID_from_path'] = final_merged_df['Image_ID'].apply(lambda x: extract_chicken_id_from_path(x))

# Merge the fused dataset with weight data based on Chicken_ID
final_with_weights_df = pd.merge(final_merged_df, weights_df[['Chicken_ID_from_path', 'weight_kg']],
                                  left_on='Chicken_ID_from_path', right_on='Chicken_ID_from_path', how='left')

# Save the final dataset with weights
final_output_with_weights_path = "/content/drive/MyDrive/final_fused_with_weights.csv"
final_with_weights_df.to_csv(final_output_with_weights_path, index=False)

print(f"Final dataset with weights saved to: {final_output_with_weights_path}")
